In [1]:
import pandas as pd
from datetime import date
from utilities.logger import FileLogger
from utilities.yahoo_data import YahooDataLoader
from model.grandma_valuation import GrandmaRegression

logger = FileLogger()
logPrint = logger.log_pandas


In [2]:
l_instrument = {
    'SP500':'IVV',
    'Greater China':'3073.HK',
    'Developed Asia': 'VPL',
    'Europe':'IEV',
    'SE Asia': 'ASEA',

    'Global Healthcare': 'IXJ',
    'Global Consumer Staple': 'KXI',
    'Global Tech': 'XT',
    'Global Consumer Discretionary':'RXI',
    'Global Finance':'IXG',

    'Asia ex Japan': 'AAXJ',
    'Eurozone':'EZU'
}

l_metrics = [] # to store the valuation metrics

for name, ticker in l_instrument.items():
    logPrint(f"{name}: {ticker}")

    # Refresh data
    yahoo = YahooDataLoader(ticker, printfunc=logPrint)
    df = yahoo.queryEOD(save=True)

    # Fit model
    grandma = GrandmaRegression(recent_months=0, train_years=10)
    df_train, df_recent = grandma.fitTransform(df, col_price='close_adj')
    d_metrics = grandma.evaluateValuation()
    df_metrics = pd.Series(d_metrics).to_frame().T
    df_metrics['ticker'] = ticker
    df_metrics['name'] = name
    l_metrics.append(df_metrics)

    # Plot image
    fig = grandma.plotTrendline(title=name)
    fig.write_image(f"output/images/{ticker}.jpeg")

    logPrint('')


SP500: IVV
IVV: Existing EOD data file found at data/IVV_EOD.csv.gz.
IVV: Existing EOD data file contains 5468 rows over 5468 dates from 2000-05-19 to 2022-02-10.
IVV: Queried EOD data contains 1 rows over 1 dates from 2022-02-10 to 2022-02-10.
IVV: Amended data file contains 5468 rows over 5468 dates from 2000-05-19 to 2022-02-10.
Train data contains 2517 rows over 2517 dates from 2012-02-13 to 2022-02-10.
No recent data specified.
Fit regression... 245 out of 2517 dates are outliers... Re-fit wihtout outliers... No recent data to estimate... done!
Train RMSE = 13.5. Annualized Return = 0.134 over 10.0 years. Compared to base price 4.09e+02, the current price 4.51e+02 is over-valued by 0.104 or 0.779 years.

Greater China: 3073.HK
3073.HK: Existing EOD data file found at data/3073.HK_EOD.csv.gz.
3073.HK: Existing EOD data file contains 2812 rows over 2812 dates from 2010-09-15 to 2022-02-11.
3073.HK: ERROR: Failed to query EOD data!
3073.HK: WARNING: No new data to refresh.
Train data

In [3]:
# Consolidate valuation metrics
df_metrics = pd.concat(l_metrics).reset_index(drop=True)

cols_first = ['ticker', 'name']
df_metrics = df_metrics[cols_first + list(df_metrics.columns.drop(cols_first))]
df_metrics.drop(columns=['rmse_train'], inplace=True)

df_metrics.to_csv(f'output/valuation_metrics_{date.today()}.csv', index=False)

df_metrics


,ticker,name,train_years,annualized_return,currenct_price,base_price,over_value_range,over_value_years
0,IVV,SP500,10.000000,0.134091,451.380005,408.714201,0.104390,0.778505
1,3073.HK,Greater China,10.002740,0.082671,51.680000,52.335771,-0.012530,-0.001036
2,VPL,Developed Asia,10.000000,0.067500,75.589996,77.025424,-0.018636,-0.001258
3,IEV,Europe,10.000000,0.052737,53.320000,49.100943,0.085926,1.629324
4,ASEA,SE Asia,10.000000,0.016286,15.770000,14.326667,0.100744,6.186115
5,IXJ,Global Healthcare,10.000000,0.110986,84.309998,82.629685,0.020335,0.183226
6,KXI,Global Consumer Staple,10.000000,0.071513,62.660000,60.784262,0.030859,0.431515
7,XT,Global Tech,6.890411,0.187436,59.349998,64.029659,-0.073086,-0.013699
8,RXI,Global Consumer Discretionary,10.000000,0.119259,166.303894,163.553163,0.016819,0.141026
9,IXG,Global Finance,10.000000,0.079702,84.900002,77.121866,0.100855,1.265395
